In [1]:
import pandas as pd
import datetime as dt
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords 
from wordcloud import WordCloud
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation as LDA
from pyLDAvis import sklearn as sklearn_lda
import re
import os
import numpy as np
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline
from nltk import tokenize
import pyLDAvis
pyLDAvis.enable_notebook()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Choice\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
lista_arquivos = os.listdir('./Dados_Valor')

In [20]:
#df_total = pd.read_hdf('./Dados_Valor/Valor_2013-01-01_2013-01-30.hdf', key = 'hdf')
for i in lista_arquivos:
    df = pd.read_hdf(f'./Dados_Valor/{i}', key = 'hdf')
    df_total = pd.concat([df_total,df], ignore_index=True)

In [21]:
df_total.shape

(166282, 3)

In [25]:
df_total.dropna()

,0,1,2
0,30/01/2016 18h03,Filho de Lula critica a imprensa e defende o p...,(Atualizada às 15h03) Fábio Luís Lula da Silva...
1,29/01/2016 18h42,COI emite alerta a federações esportivas sobre...,O Comitê Olímpico Internacional (COI) enviou u...
2,29/01/2016 08h00,Cardápio renovado para atrair clientes,Entrar\nAssinar
3,29/01/2016 08h00,Mestres em transformação,"""Só há duas coisas certas na vida: a morte e o..."
4,29/01/2016 08h00,Lugar de estudante é o que tem boa conexão de ...,"A velha máxima de que ""lugar de estudante é na..."
5,29/01/2016 08h00,Estante do empreendedor,
6,29/01/2016 08h00,Petrobras se reestrutura em busca de economia ...,
7,29/01/2016 08h00,O homem que falava 'djavanês',
8,29/01/2016 08h00,Técnica dosa conhecimento em pílulas,
9,29/01/2016 08h00,Livros que estimulam as discussões na sala de ...,
